In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import requests

# As there are possibilities of different chrome
# browser and we are not sure under which it get
# executed let us use the below syntax
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/sgo/.wdm/drivers/chromedriver/mac64_m1/102.0.5005.61/chromedriver] found in cache
/var/folders/j5/d8bm9_0n5rv71d349szjs_980000gn/T/ipykernel_27934/2159926137.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [2]:
# from selenium import webdriver
# driver = webdriver.Chrome()
#London Victoria & Albert Museum URL
# url = 'https://www.google.com/maps/place/Victoria+and+Albert+Museum/@51.4966392,-0.17218,15z/data=!4m5!3m4!1s0x0:0x9eb7094dfdcd651f!8m2!3d51.4966392!4d-0.17218'

# search = 'Pinakothek der Moderne'
# search = 'SEEN RESTAURANT'
search = 'Ya The Mandarin Room'
search = '+'.join(search.split(' '))

url = f'https://www.google.com/maps/place/{search}'
driver.get(url)
url

'https://www.google.com/maps/place/Ya+The+Mandarin+Room'

In [3]:
driver.find_element(by=By.XPATH, 
                    value='//*[@id="searchbox-searchbutton"]').click()
time.sleep(2)

In [4]:
driver.find_element(by=By.CLASS_NAME, 
                    value='DkEaL').click()
time.sleep(2)

In [5]:
# try:
#     driver.find_element(By.CLASS_NAME, "widget-pane-link").click()
# except Exception:
#     response = BeautifulSoup(driver.page_source, 'html.parser')
#     # Check if there are any paid ads and avoid them
#     if response.find_all('span', {'class': 'ARktye-badge'}):
#         ad_count = len(response.find_all('span', {'class': 'ARktye-badge'}))
#         li = driver.find_elements(By.CLASS_NAME, "a4gq8e-aVTXAb-haAclf-jRmmHf-hSRGPd")
#         li[ad_count].click()
#     else:
#         driver.find_element(By.CLASS_NAME, "a4gq8e-aVTXAb-haAclf-jRmmHf-hSRGPd").click()
#         time.sleep(5)
#     driver.find_element(By.CLASS_NAME, "widget-pane-link").click()

In [6]:
total_number_of_reviews = driver.find_element(By.XPATH, "//*[@class='fontBodySmall']").text.split(" ")[0]
total_number_of_reviews = int(total_number_of_reviews.replace('.','')) if '.' in total_number_of_reviews else int(total_number_of_reviews)

total_number_of_reviews

430

In [7]:
#Find scroll layout
scrollable_div = driver.find_element(By.XPATH, "//*[@id='QA0Szd']/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]")

In [8]:
count = 0
#Scroll as many times as necessary to load all reviews
for i in range(0,(round(total_number_of_reviews/10 - 1))):
    more = driver.find_elements(by=By.XPATH, 
                    value='//*[@aria-label="Mehr"]')
    for m in more:
        m.click()
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', 
            scrollable_div)
    time.sleep(2)
    count+=1
    if 2*count > 300: break

In [9]:
response = BeautifulSoup(driver.page_source, 'html.parser')
texts = response.find_all('span', class_='wiI7pd')
rates = response.find_all('div', class_='DU9Pgb')

In [10]:
def get_review_summary(text_set, rate_set):
    rev_dict = {'Review Text': [],
               'Review Rate': [],
               'Review Time': []}
    for text, rate in zip(text_set, rates):
        review_rate = rate.find('span', class_='kvMYJc')["aria-label"].strip()[0]
        review_time = rate.find('span',class_='rsqaWe').text
        review_text = text.text
        rev_dict['Review Rate'].append(review_rate)
        rev_dict['Review Time'].append(review_time)
        rev_dict['Review Text'].append(review_text)
    import pandas as pd    
    return(pd.DataFrame(rev_dict))

df = get_review_summary(texts, rates)

In [11]:
from datetime import date

df['search']=search
df['insert_date']=date.today()

df

,Review Text,Review Rate,Review Time,search,insert_date
0,Der erste Eindruck lässt einem schon direkt de...,5,vor 5 Monaten,Ya+The+Mandarin+Room,2022-06-26
1,"Sehr gutes asiatisches Restaurant, wo man in M...",5,vor einer Woche,Ya+The+Mandarin+Room,2022-06-26
2,Sehr schön eingerichtetes Restaurant! Service ...,5,vor 4 Monaten,Ya+The+Mandarin+Room,2022-06-26
3,Die Atmosphäre in dem Restaurant war sehr cool...,5,vor einem Monat,Ya+The+Mandarin+Room,2022-06-26
4,Waren am Sonntag Abend das erste Mal im Ya und...,5,vor einem Monat,Ya+The+Mandarin+Room,2022-06-26
...,...,...,...,...,...
425,,5,vor 10 Monaten,Ya+The+Mandarin+Room,2022-06-26
426,,5,vor 3 Monaten,Ya+The+Mandarin+Room,2022-06-26
427,,5,vor einem Monat,Ya+The+Mandarin+Room,2022-06-26
428,,5,vor 10 Monaten,Ya+The+Mandarin+Room,2022-06-26


In [12]:

today = date.today()
print("Today's date:", today)

Today's date: 2022-06-26


In [13]:
df.to_csv(f'{search}.csv', index=False)

In [14]:
f'{search}.csv'
driver.close()